## Assigment: Neural network basics

Soft deadline: 16.09.18 at 23.59

Hard deadline: 18.09.18 at 23.59

For this task I intentionally provide no boilerplate code, because very puprpose of this task is getting you comforatable with basic code template for desiging NNs in pytorch. I higly recommend you to revisit all the last seminar materials.

#### Task

* Implement simple **fully-convolutional** neural architecture for classification. Make sure it is small enought to run on your home machine.
* Provide dataset visulization.
* Provide train/test split and validation

#### Requirements

* Architecture should derive from `torch.nn.Module`
* Use `torch.utils.data.Dataset` and `torch.utils.data.DataLoader`. But if you manage co simplify this step using dataset `torchivision`, I will only encourage you.
* Implement at least one data transformer, but make sure it is useful for classification task.
* Use FashionMNIST dataset https://github.com/zalandoresearch/fashion-mnist
* Make sure you can fix random seed for all components of your code to make experiments reproducible
* Since you architecure should be fully-convolutional, make sure it does not depend on input size.

In [1]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

batch_size = 100
learing_rate = 0.01

In [2]:
dataset_dir = "fasion_mnist"
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
train_dataset = datasets.FashionMNIST(dataset_dir, train=True, download=False, transform=transform)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.FashionMNIST(dataset_dir, train=False, download=False, transform=transform)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
i = 0
for a, _ in train_dataset:
    i += 1
    if i > 3:
        break
    print(a)

tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1

In [5]:
class Net(nn.Module):
    def __init__(self, size_n=28, size_m=28, classes=10):
        super(Net, self).__init__()
        kernel = 5
        conv_channels_1 = 4
        conv_channels_2 = 10
        hidden_layer_size = 15
        max_pool_size = 2
        
        self.size_n = size_n
        self.size_m = size_m
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, conv_channels_1, kernel_size=kernel),
            nn.MaxPool2d(max_pool_size)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(conv_channels_1, conv_channels_2, kernel_size=5),
            nn.MaxPool2d(max_pool_size)
        )
        size_n = (size_n - kernel + 1) // max_pool_size
        size_n = (size_n - kernel + 1) // max_pool_size
        
        size_m = (size_m - kernel + 1) // max_pool_size
        size_m = (size_m - kernel + 1) // max_pool_size
        
        self.lin1 = nn.Linear(size_n * size_m * conv_channels_2, hidden_layer_size)
        self.lin2 = nn.Linear(hidden_layer_size, classes)
            
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, x[0].numel())
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

net = Net()

In [14]:
iters = 12

def train():
    model = net.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learing_rate)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(1, iters + 1):
        for barch_id, (image, label) in enumerate(train_dataloader):
            label, image = label.to(device), image.to(device)
            output = model(image)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if barch_id % 1000 == 0:
                print('Loss :{:.4f} Epoch[{}/{}]'.format(loss.item(), epoch, iters))
                print("output {}, class = {}".format(output[0], label[0]))
                print()
    
    return model

model = train()



Loss :0.2267 Epoch[1/12]
output tensor([  -5.4231,   13.8346,  -13.8492,   -0.1907,   -5.0315,  -48.7943,
          -8.6751, -119.0128,  -19.3955,  -61.2973], grad_fn=<SelectBackward>), class = 1



Loss :0.4204 Epoch[2/12]
output tensor([ -1.7768,  -8.8237,   2.2706,   0.5468,   7.3278, -30.1592,   5.2356,
        -82.9666,  -8.9607, -28.4215], grad_fn=<SelectBackward>), class = 4



Loss :0.3033 Epoch[3/12]
output tensor([-21.0391, -30.4888, -21.3245, -13.9192, -21.8676,   9.0385, -21.4027,
          9.3195, -21.0452,  17.0950], grad_fn=<SelectBackward>), class = 9



Loss :0.4166 Epoch[4/12]
output tensor([ -6.2629,  -6.8157,  -7.1801,   1.7272,  -5.2375, -23.6075,  -5.4261,
        -32.3824,  -6.8254, -30.4557], grad_fn=<SelectBackward>), class = 3



Loss :0.4347 Epoch[5/12]
output tensor([ -9.6818, -80.8140, -30.9937, -12.8696, -70.1078,  -1.1379, -34.2184,
          4.7902,  -3.1786,   0.9564], grad_fn=<SelectBackward>), class = 7



...

Loss :0.2978 Epoch[8/12]
output tensor([ -5.0501,  -6.0155,  -9.0516, -10.2223,  -7.6716,  -7.8754,  -6.6119,
         -9.2585,   0.7243,  -3.9627], grad_fn=<SelectBackward>), class = 8



Loss :0.3561 Epoch[9/12]
output tensor([ -11.3860, -108.1257,  -33.0875,  -14.3126,  -93.8093,    2.1155,
         -45.5865,    6.6427,   -5.6213,    3.5228], grad_fn=<SelectBackward>), class = 7



Loss :0.3425 Epoch[10/12]
output tensor([ -6.1941,   2.1166,  -6.3468,  -5.9938,  -8.2885, -13.9411,  -8.2050,
        -83.7117, -16.3695,  -8.0031], grad_fn=<SelectBackward>), class = 1



Loss :0.1825 Epoch[11/12]
output tensor([ -4.1333,  -4.5839,  -7.9067,   2.7083,  -4.7387, -19.9461,  -5.0546,
        -23.6577,  -6.1365, -26.9207], grad_fn=<SelectBackward>), class = 3



Loss :0.2576 Epoch[12/12]
output tensor([ -13.8051, -107.7575,  -33.1344,  -10.4387,  -96.4540,    0.8927,
         -50.1888,    6.2305,   -8.0660,    4.3104], grad_fn=<SelectBackward>), class = 7



In [15]:
def test(model):
    with torch.no_grad():
        correct = 0
        total = 0
        for image, label in test_dataloader:
            image = image.to(device)
            label = label.to(device)
            outputs = model(image)
            predicted = torch.argmax(outputs,dim=1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
        print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))
test(model)

Test Accuracy of the model on the test images: 85.65 %
